In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import checklist
import spacy
import itertools

In [404]:
import checklist.editor
import checklist.text_generation
from checklist.mft import Mft
from checklist.inv_dir import Inv, Dir
from checklist.expect import Expect

In [8]:

nlp = spacy.load('en_core_web_sm')

In [9]:
tg = checklist.text_generation.TextGenerator(nlp)

In [35]:
from checklist.pred_wrapper import PredictorWrapper

In [161]:
import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
sentiment = model_wrapper.ModelWrapper()
pp = PredictorWrapper.wrap_softmax(sentiment.predict_proba)

In [10]:
# tg.unmask('I don\'t know <mask> I am going to cope.')
[x[0][0] for x in tg.unmask('I really <mask> the pilot.')]

['liked',
 'like',
 'enjoyed',
 'loved',
 'admired',
 'love',
 'admire',
 'enjoy',
 'appreciated',
 'appreciate',
 'respect',
 'was',
 'disliked',
 'miss',
 'respected',
 'likes',
 'trust',
 'missed',
 'hated',
 'wanted']

In [11]:
a = tg.unmask('I really <mask> the pilot.')
b = tg.unmask('I really <mask> the flight.')

In [12]:
# tg.unmask_multiple(['I really <mask> the pilot.', 'I really <mask> the flight.'], metric='min')

In [202]:
editor = checklist.editor.Editor()
editor.lexicons['male'] = ['John', 'Luke', "Mark"]
editor.lexicons['female'] = ['Mary', 'Judy', "Julia"]

In [15]:
nouns = ['flight', 'seat', 'pilot', 'staff', 'plane', 'airline', 'cabin crew', 'aircraft', 'food']
pos = ['liked', 'enjoyed', 'loved', 'admired', 'appreciated']
neg = ['hated', 'disliked', 'regretted']


In [16]:
ts = editor.template('I really <mask> the {n}.', n=nouns)
print([x[0][0] for x in tg.unmask_multiple(ts, beam_size=1000)][:20])
# tg.unmask_multiple(ts, metric='avg', beam_size=100)

['like', 'liked', 'love', 'enjoyed', 'loved', 'appreciate', 'enjoy', 'appreciated', 'miss', 'Like', 'missed', 'needed', 'hate', 'wanted', 'likes', 'got', 'admired', 'admire', 'hated', 'recommend']


In [17]:
ts = editor.template('<mask> really {pn} the {n}.', n=nouns, pn=pos+neg)
print([x[0][0] for x in tg.unmask_multiple(ts, beam_size=1000)][:20])
# tg.unmask_multiple(ts, metric='avg', beam_size=100)

['I', 'We', 'He', 'he', 'She', 'They', 'we', 'and', '...', 'she', 'they', 'Everyone', 'i', 'People', 'You', 'Alex', 'Everybody', 'And', 'Dad', 'John']


In [18]:
subj = ['I', 'We', 'They', 'we', 'He', 'he', 'She', 'she', 'they', 'people', 'People', 'you', '']

In [19]:
ts = editor.template('I didn\'t like the {n}, it was very <mask>.', n=nouns)
print([x[0][0] for x in tg.unmask_multiple(ts, beam_size=1000, metric='avg')][:100])
# tg.unmask_multiple(ts, metric='avg', beam_size=100)

['bad', 'boring', 'strange', 'cold', 'slow', 'unpleasant', 'weird', 'difficult', 'uncomfortable', 'expensive', 'rough', 'noisy', 'ugly', 'poor', 'dark', 'different', 'small', 'disappointing', 'quiet', 'rude', 'confusing', 'dull', 'sad', 'odd', 'loud', 'annoying', 'simple', 'harsh', 'short', 'unusual', 'scary', 'hard', 'nice', 'hot', 'dangerous', 'aggressive', 'heavy', 'dirty', 'conservative', 'old', 'tough', 'bland', 'low', 'cheap', 'frustrating', 'depressing', 'nasty', 'long', 'crude', 'horrible', 'stupid', 'complicated', 'funny', 'fast', 'good', 'basic', 'weak', 'wrong', 'awkward', 'terrible', 'negative', 'crowded', 'plain', 'thin', 'interesting', 'awful', 'bizarre', 'intimidating', 'cool', 'primitive', 'unstable', 'flat', 'narrow', 'stressful', 'stiff', 'empty', 'chaotic', 'light', 'irritating', 'big', 'unfair', 'sick', 'similar', 'angry', 'mean', 'predictable', 'easy', 'offensive', 'foreign', 'violent', 'tight', 'comfortable', 'busy', 'challenging', 'arrogant', 'disgusting', 'ineff

In [20]:
pos_adj = ['good', 'nice', 'helpful', 'comfortable', 'cool', 'efficient', 'pleasant', 'interesting', 'impressive', 'welcoming', 'professional', 'beautiful', 'exciting', 'positive', 'solid', 'amazing', 'wonderful', 'lovely']
neg_adj = ['bad', 'boring', 'unpleasant', 'difficult', 'uncomfortable', 'ugly', 'poor', 'disappointing', 'sad', 'annoying', 'dirty', 'frustrating', 'depressing', 'nasty', 'horrible', 'stupid', 'negative', 'awful', 'stressful', 'irritating', 'disgusting']

In [21]:
pos_eg = editor.template('{sub}{really} {p} the {n}.', sub=subj, really=[' really', ''], n=nouns, p=pos)
pos_eg += editor.template('The {n} was{very} {p}.', sub=subj, very=[' very', ''], n=nouns, p=pos_adj)
neg_eg = editor.template('{sub}{really} {p} the {n}.', sub=subj, really=[' really', ''], n=nouns, p=neg)
neg_eg += editor.template('The {n} was{very} {p}.', sub=subj, very=[' very', ''], n=nouns, p=neg_adj)

In [185]:
len(pos_eg), len(neg_eg)

(1494, 1080)

In [255]:
test_pos = Mft(pos_eg, labels=1)
test_neg = Mft(neg_eg, labels=0)

In [187]:
test_pos.run(pp)
test_neg.run(pp)

In [190]:
test_pos.results.passed.mean(), test_neg.results.passed.mean()

(1.0, 0.9833333333333333)

In [199]:
for f in np.where(test.results.passed == False)[0]:
    print(test.results.confs[f][1].round(3), test.data[f])

0.999 The flight was very stressful.
0.995 The seat was very stressful.
0.999 The pilot was very stressful.
1.0 The staff was very stressful.
0.999 The plane was very stressful.
0.998 The airline was very stressful.
0.998 The cabin crew was very stressful.
0.999 The aircraft was very stressful.
0.999 The food was very stressful.
0.999 The flight was stressful.
0.963 The seat was stressful.
0.997 The pilot was stressful.
0.999 The staff was stressful.
0.999 The plane was stressful.
0.97 The airline was stressful.
0.994 The cabin crew was stressful.
0.998 The aircraft was stressful.
0.997 The food was stressful.


## Inv, dir

In [200]:
import csv
r = csv.DictReader(open('/home/marcotcr/datasets/airline/Tweets.csv'))
labels = []
confs = []
airlines = []
tdata = []
reasons = []
for row in r:
    sentiment, conf, airline, text = row['airline_sentiment'], row['airline_sentiment_confidence'], row['airline'], row['text']
    labels.append(sentiment)
    confs.append(conf)
    airlines.append(airline)
    tdata.append(text)
    reasons.append(row['negativereason'])

mapping = {'negative': 0, 'positive': 2, 'neutral': 1}
labels = np.array([mapping[x] for x in labels]).astype(int)

In [201]:
sentences = tdata
parsed_data = list(nlp.pipe(sentences))

In [240]:
def add_typo(string):
    string = list(string)
    swaps = 1
    swaps = np.random.choice(len(string) - 1, swaps)
    for swap in swaps:
        swap = np.random.choice(len(string) - 1)
        tmp = string[swap]
        string[swap] = string[swap + 1]
        string[swap + 1] = tmp
    return ''.join(string)

def add_typos(string):
    return list(set([add_typo(string) for _ in range(10)]))


In [233]:
from checklist.perturb import Perturb

In [400]:
data = Perturb.perturb(np.random.choice(sentences, 100), add_typos, keep_original=True)

In [401]:
test = Inv(data, threshold=0.1)
test.run(pp, overwrite=True)

In [402]:
test.results.passed.mean()

0.77

In [403]:
for f in np.where(test.results.passed == False)[0][:5]:
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: np.abs(x[1] - dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.2f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

0.01 @USAirways I hope flight 1855 is not being delayed because flight attendant didn't come to work! #nobackup
0.96 @USAirways Ih ope flight 1855 is not being delayed because flight attendant didn't come to work! #nobackup
0.00 @USAirways I hope flight 1855 is not beingd elayed because flight attendant didn't come to work! #nobackup

0.01 @JetBlue's CEO battles to appease passengers and Wall Street - Waterbury Republican American http://t.co/Of7PFvQpOY
0.95 @JetBlue's CEOb attles to appease passengers and Wall Street - Waterbury Republican American http://t.co/Of7PFvQpOY
0.91 @JetBlue's CEO battlest o appease passengers and Wall Street - Waterbury Republican American http://t.co/Of7PFvQpOY

0.95 @JetBlue ETA for flight 802 into Orlando this morning
0.04 @JetBlue ETA for flight 802 into Orlando this mornnig
0.29 @eJtBlue ETA for flight 802 into Orlando this morning

0.27 @united I think you should board from the back of the plane #whatstheholdup #CHItoCLE
0.92 @united I think you shoul

In [399]:
data = Perturb.perturb(np.random.choice(sentences, 1000), add_typo, keep_original=True)
test = Inv(data, threshold=0.1)
test.run(pp, overwrite=True)

In [395]:
test.results.passed.mean()

0.954

In [396]:
for f in np.where(test.results.passed == False)[0][:5]:
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: np.abs(x[1] - dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.2f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

0.42 @AmericanAir it wasn't ' disrupted' it was Cancelled Flightled. Airport agents were horrendous. Sharon was your saviour
0.99 @AmericanAir it wasn't ' disrupted' it was Cancelled Flightled. Airport agents were horrenduos. Sharon was your saviour

0.80 @JetBlue gr8 #Mint crew on #flight 123 to #LAX they're #Mintalicious #TrueBlueLove #ShelleyandMarcRock #travel #air
0.20 @JetBlue gr8 #Mint crew on #flight 123 to #LAX theyr'e #Mintalicious #TrueBlueLove #ShelleyandMarcRock #travel #air

0.16 Need more info on that! @united: @momsgoodeats We have great amenity kits. PJ's not included. We status match AA. ^KP”
0.85 Need more info on that! @united: @momsgoodeats We have great amenity kits. P'Js not included. We status match AA. ^KP”

0.08 @AmericanAir can you guys help me please?
0.78 @AmericanAir can you guys help em please?

0.10 @USAirways I finally got through on the phone after a 2 hour wait time.
0.55 @USAirways I finally got through on th ephone after a 2 hour wait time.



In [433]:
# TODO: get from somethwere else
def add_negatives(string):
    string = string.strip('.')
    return [string + '. ' + l for l in ['I hate you', 'I despise you', 'You suck']]
def add_positive(string):
    string = string.strip('.')
    return [string + '. ' + l for l in ['I love you', 'I like you', 'You are great']]


In [412]:
add_negatives(sentences[0])

['@VirginAmerica What @dhepburn said. I hate you',
 '@VirginAmerica What @dhepburn said. I despise you',
 '@VirginAmerica What @dhepburn said. You suck']

In [432]:
data = Perturb.perturb(np.random.choice(sentences, 100), add_negatives, keep_original=True)
test = Dir(data, Expect.wrap(Expect.all(Expect.pairwise_to_group(Expect.monotonic_label(1, increasing=False, tolerance=0)))) )
test.run(pp, overwrite=True)
test.results.passed.mean()

In [431]:
for f in np.where(test.results.passed == False)[0][:5]:
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: (x[1] - dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.5f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

0.00088 @USAirways thank you for blowing my vacation. Couldn't get me anywhere today to make my reservation and also lost 2 bags of mine!
0.00081 @USAirways thank you for blowing my vacation. Couldn't get me anywhere today to make my reservation and also lost 2 bags of mine!. You suck
0.00079 @USAirways thank you for blowing my vacation. Couldn't get me anywhere today to make my reservation and also lost 2 bags of mine!. I despise you

0.00433 @united I will but right now I'm to angry
0.00239 @united I will but right now I'm to angry. I despise you
0.00170 @united I will but right now I'm to angry. I hate you

0.11484 @united you should tell that to the staff at LAX then. I boarded group 5 at the very end of the queue as a Gold member. Thanks for nothing.
0.01019 @united you should tell that to the staff at LAX then. I boarded group 5 at the very end of the queue as a Gold member. Thanks for nothing. You suck
0.00678 @united you should tell that to the staff at LAX then. I boarded grou

In [434]:
data = Perturb.perturb(np.random.choice(sentences, 100), add_positive, keep_original=True)
test = Dir(data, Expect.wrap(Expect.all(Expect.pairwise_to_group(Expect.monotonic_label(1, increasing=True, tolerance=0)))) )
test.run(pp, overwrite=True)
test.results.passed.mean()

1.0

In [436]:
test.results

Munch({'preds': [array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 0, 1, 1]), array([1, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 0, 0, 1]), array([1, 1, 1, 1]), array([0, 0, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([1, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([1, 1, 1, 1]), array([0, 0, 0, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([1, 1, 1, 1]), array([0, 0, 1, 1]), array([1, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 0, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([1, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 0, 1, 1]), array([1, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([1, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 0, 0, 0]), array([1, 1, 1, 1]), array([1, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([1, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1]), array([0, 1, 1, 1

In [435]:
for f in np.where(test.results.passed == False)[0][:5]:
    dataconfs = list(zip(test.data[f], test.results.confs[f][:, 1]))
    dataconfs[1:] = sorted(dataconfs[1:], key=lambda x: (- x[1] + dataconfs[0][1]), reverse=True)
    print('\n'.join(['%.5f %s' % (x[1], x[0]) for x in dataconfs][:3]))
    print()

In [13]:
editor.template('This is {bad}',  bad=['bad', 'great', 'awesome'], return_maps=True)

(['This is bad', 'This is great', 'This is awesome'],
 [{'bad': 'bad'}, {'bad': 'great'}, {'bad': 'awesome'}])

In [14]:

editor.template(('This is {bad}', 'This is not {bad}'),  bad=['bad', 'great', 'awesome'], return_maps=False)

([('This is bad', 'This is not bad'),
  ('This is great', 'This is not great'),
  ('This is awesome', 'This is not awesome')],
 [{'bad': 'bad'}, {'bad': 'great'}, {'bad': 'awesome'}])

In [15]:
editor.template({
    'bad': 'This is {bad}',
    'notbad': 'This is not {bad}'},  bad=['bad', 'great', 'awesome'], return_maps=False)

[{'bad': 'This is bad', 'notbad': 'This is not bad'},
 {'bad': 'This is great', 'notbad': 'This is not great'},
 {'bad': 'This is awesome', 'notbad': 'This is not awesome'}]

In [57]:
a = {'bad': 'This is {bad}',
    'notbad': ('this is not {bad}', 'this is quite {abad}')}
b = ({'bad': 'This is {bad}', 'notbad': 'This is not {bad}'}, 'This is quite {bad}')
c = 'This is quite {bad}.'

In [64]:
editor.lexicons

{'male': ['John', 'Luke', 'Mark'], 'female': ['Mary', 'Judy', 'Julia']}

In [93]:
editor.template('This is {bad}, {male}', bad=('terrible', 'bad'), nsamples=None)

['This is terrible, John',
 'This is bad, John',
 'This is terrible, Luke',
 'This is bad, Luke',
 'This is terrible, Mark',
 'This is bad, Mark']